# Day 53 – Model Tuning and Feature Selection

This notebook demonstrates grid search, Bayesian optimisation, and feature selection utilities.

In [ ]:

from Day_53_Model_Tuning_and_Feature_Selection.solutions import (
    compute_permutation_importance,
    evaluate_selected_features,
    generate_tuning_dataset,
    run_bayesian_optimisation,
    run_grid_search,
    run_recursive_feature_elimination,
)

X, y = generate_tuning_dataset()
grid_result = run_grid_search(X, y)
bayes_result = run_bayesian_optimisation(X, y, n_iter=10)
grid_result.best_params, round(grid_result.best_score, 3), round(bayes_result.best_score, 3)


Inspect permutation importance and recursive feature elimination.

In [ ]:

best_estimator = grid_result.search.best_estimator_
permutation_df = compute_permutation_importance(best_estimator, X, y)
selector, support = run_recursive_feature_elimination(X, y)
selected_score = evaluate_selected_features(selector, X, y)
permutation_df.head(), support, round(selected_score, 3)
